## 1. 기상데이터 및 가격데이터 로드 및 SQL 연동
- 기상청자료개방포털에서 다운받은 국내 및 해외 날씨자료 read
- KAMIS에 API 요청을 통해 농산물 별 가격정보 가져오기
- 농산물 가격정보를 MySQL DB와 연동

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os

import mysql.connector
import requests
import json

## 기상데이터 로드

#### csv, xls 파일로 저장된 기후 데이터 경로 가져오기

In [2]:
def is_sheet(name) :
    return ('.csv' in name) or ('.xls' in name)

In [3]:
climate_paths = {
    '고추' : [],
    '마늘' : [],
    '쌀' : [],
    '양파' : [],
    '배추' : [],
    '양배추' : [],
    '토마토' : [],
    '무' : [],
    '시금치' : [],
    '당근' : []
}
climate_paths = {
    '상추' : [],
}
for vege in climate_paths.keys() :
    base_path = [f'./data/climate_seperated/{vege}']

    while True :
        deep_path = []

        for path in base_path:
            dir_list = os.listdir(path)

            for name in dir_list :
                if is_sheet(name) :
                    temp_path = f'{path}/{name}'
                    climate_paths[vege].append(temp_path)
                else :
                    if (name.lower() == 'desktop.ini') or (name.lower() == '.ds_store') :
                        pass
                    else :
                        temp_path = f'{path}/{name}'
                        deep_path.append(temp_path)

        if len(deep_path) == 0 :
            break
        else :
            base_path = deep_path

In [4]:
climate_paths['상추']

['./data/상추/SURFACE_ASOS_114_HR_2010_2010_2015.csv',
 './data/상추/SURFACE_ASOS_114_HR_2011_2011_2015.csv',
 './data/상추/SURFACE_ASOS_114_HR_2011_2020_2021.csv',
 './data/상추/SURFACE_ASOS_114_HR_2012_2012_2015.csv',
 './data/상추/SURFACE_ASOS_114_HR_2013_2013_2015.csv',
 './data/상추/SURFACE_ASOS_114_HR_2014_2014_2015.csv',
 './data/상추/SURFACE_ASOS_114_HR_2015_2015_2016.csv',
 './data/상추/SURFACE_ASOS_114_HR_2016_2016_2017.csv',
 './data/상추/SURFACE_ASOS_114_HR_2017_2017_2018.csv',
 './data/상추/SURFACE_ASOS_114_HR_2018_2018_2019.csv',
 './data/상추/SURFACE_ASOS_114_HR_2019_2019_2020.csv',
 './data/상추/SURFACE_ASOS_114_HR_2020_2020_2021.csv',
 './data/상추/SURFACE_ASOS_203_HR_2010_2010_2015.csv',
 './data/상추/SURFACE_ASOS_203_HR_2011_2011_2015.csv',
 './data/상추/SURFACE_ASOS_203_HR_2011_2020_2021.csv',
 './data/상추/SURFACE_ASOS_203_HR_2012_2012_2015.csv',
 './data/상추/SURFACE_ASOS_203_HR_2013_2013_2015.csv',
 './data/상추/SURFACE_ASOS_203_HR_2014_2014_2015.csv',
 './data/상추/SURFACE_ASOS_203_HR_2015_2015_2016

#### 가져온 경로를 바탕으로 채소별로 하나의 csv 파일로 정리.

In [5]:
df_na = {}

df_for = {}

for vege in climate_paths.keys():
    national = []
    foreign = []
    
    for path in climate_paths[vege]:
        national.append(pd.read_csv(path, encoding = 'cp949'))
            
    if len(national) != 0 :
        df_na[vege] = pd.concat(national)
    if len(foreign) != 0 :
        df_for[vege] = pd.concat(foreign)

C:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [33]:
for vege in climate_paths.keys() :
    df_na[vege].to_csv(f'./data/climate_merged/{vege}_na.csv', encoding = 'cp949')
    df_for[vege].to_csv(f'./data/climate_merged/{vege}_for.csv', encoding = 'cp949')

In [22]:
df_na['고추']

,지점,지점명,일시,기온(°C),기온 QC플래그,강수량(mm),강수량 QC플래그,풍속(m/s),풍속 QC플래그,풍향(16방위),...,최저운고(100m ),시정(10m),지면상태(지면상태코드),현상번호(국내식),지면온도(°C),지면온도 QC플래그,5cm 지중온도(°C),10cm 지중온도(°C),20cm 지중온도(°C),30cm 지중온도(°C)
0,136,안동,2021-01-01 00:00,-7.0,NaN,NaN,NaN,3.9,NaN,290.0,...,NaN,2000.0,NaN,NaN,-8.9,NaN,NaN,NaN,NaN,NaN
1,136,안동,2021-01-01 01:00,-7.1,NaN,NaN,NaN,3.0,NaN,290.0,...,NaN,2000.0,NaN,NaN,-9.4,NaN,NaN,NaN,NaN,NaN
2,136,안동,2021-01-01 02:00,-7.2,NaN,NaN,NaN,2.9,NaN,290.0,...,NaN,2000.0,NaN,NaN,-9.4,NaN,NaN,NaN,NaN,NaN
3,136,안동,2021-01-01 03:00,-7.5,NaN,NaN,NaN,3.0,NaN,290.0,...,NaN,2000.0,NaN,NaN,-9.7,NaN,NaN,NaN,NaN,NaN
4,136,안동,2021-01-01 04:00,-7.5,NaN,NaN,NaN,3.1,NaN,290.0,...,NaN,2000.0,NaN,NaN,-10.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8779,221,NaN,2020-12-31 19:00,-7.5,NaN,NaN,NaN,2.3,NaN,230.0,...,7.0,2000.0,NaN,NaN,-5.6,NaN,NaN,NaN,NaN,NaN
8780,221,NaN,2020-12-31 20:00,-8.2,NaN,NaN,NaN,2.0,NaN,230.0,...,NaN,2000.0,NaN,NaN,-7.1,NaN,NaN,NaN,NaN,NaN
8781,221,NaN,2020-12-31 21:00,-8.7,NaN,NaN,NaN,1.5,NaN,270.0,...,NaN,1943.0,NaN,NaN,-7.9,NaN,NaN,NaN,NaN,NaN
8782,221,NaN,2020-12-31 22:00,-8.6,NaN,NaN,NaN,2.1,NaN,270.0,...,NaN,1998.0,NaN,NaN,-8.4,NaN,NaN,NaN,NaN,NaN


## 농작물 가격데이터 로드

#### API 호출을 통해서 작물별 가격데이터 가져오기

In [11]:
class Metadatas :
    def __init__(self, category, item, kind) :
        self.category = category,
        self.item = item,
        self.kind = kind,

vege_metadata = {
    '쌀' : Metadatas('100', '111', [('01', '전체')]),
    '배추' : Metadatas('200', '211', [('01', '봄'), ('02', '고랭지'), ('03', '가을'), ('06', '월동')]),
    '양배추' : Metadatas('200', '212', [('00', '전체')]),
    '시금치' : Metadatas('200', '213', [('00', '전체')]),
    '토마토' : Metadatas('200', '225', [('00', '전체')]),
    '무' : Metadatas('200', '231', [('01', '봄'), ('02', '고랭지'), ('03', '가을')]),
    '당근' : Metadatas('200', '232', [('01', '전체')]),
    '건고추' : Metadatas('200', '241', [('00', '화건'), ('01', '햇살화건'), ('02', '양건'), ('03', '햇살양건')]),
    '풋고추' : Metadatas('200', '242', [('00', '풋고추'), ('03', '청양고추')]),
    '홍고추' : Metadatas('200', '243', [('00', '홍고추')]),
    '피마늘' : Metadatas('200', '244', [('03', '한지'), ('04', '난지')]),
    '양파' : Metadatas('200', '245', [('00', '양파'), ('02', '햇양파')]),
}

In [12]:
dates = [
    ('2010-01-01', '2010-12-31'),
    ('2011-01-01', '2011-12-31'),
    ('2012-01-01', '2012-12-31'),
    ('2013-01-01', '2013-12-31'),
    ('2014-01-01', '2014-12-31'),
    ('2015-01-01', '2015-12-31'),
    ('2016-01-01', '2016-12-31'),
    ('2017-01-01', '2017-12-31'),
    ('2018-01-01', '2018-12-31'),
    ('2019-01-01', '2019-12-31'),
    ('2020-01-01', '2020-12-31'),
    ('2021-01-01', '2021-10-15'),
]

In [47]:
def df_transform(raw_df, item, kind, rank):
    raw_df['date'] = raw_df['yyyy'] + '/' + raw_df['regday']
    raw_df['price'] = raw_df.price.apply(lambda x : x.replace(',',''))
    date = pd.to_datetime(raw_df[raw_df['countyname'] == '평년'].date)
    price = pd.to_numeric(raw_df[raw_df['countyname'] == '평년'].price)
    df = pd.DataFrame(price).set_index(date)
    df['item'] = item
    df['kind'] = kind
    df['ranks'] = rank
    df['county'] = '평년'
    return df

In [48]:
for k, v in vege_metadata.items() :
    for date in dates :
        url = 'http://www.kamis.or.kr/service/price/xml.do?action=periodProductList'
        api_id = '2112'
        api_key = 'ad490b51d-fa82-4b50-99d1-68a3d9311cdc'
        return_type = 'json'
        start_date = date[0]
        end_date = date[1]
        product_code = '02' # 도매
        item_category = v.category
        item_code = v.item
        for kin in v.kind[0] :
            kind_code = kin[0]
            rank_code = '04'

            params = {
                'p_cert_key':api_key,
                'p_cert_id':api_id,
                'p_returntype':return_type,
                'p_startday':start_date,
                'p_endday':end_date,
                'p_itemcategorycode':item_category,
                'p_itemcode':item_code,
                'p_kindcode':kind_code,
                'p_productrankcode':rank_code,
                'p_productclscode':product_code,
                'p_countrycode':'1101'
            }

            try :
                res = requests.get(url, params)
                data = json.loads(res.text)
                if data['data']['error_code'] == '000' :
                    raw_df = pd.DataFrame(data['data']['item'])
                    df = df_transform(raw_df, k, kin[1], '상품')
                    api_to_db(df)
            except Exception as e:
                print(e)
            

got db!
got db!
got db!
got db!
got db!
got db!
got db!
got db!
got db!
got db!
got db!
got db!
got db!
got db!
got db!
got db!
got db!
got db!
got db!
got db!
got db!
got db!
got db!
got db!


In [51]:
db.commit()

## MySQL 연동

#### MySQL 연동을 위한 코드작성

In [16]:
class MySQLDBCrud:
    
    def __init__(self, host, port, user, password, db_name, auto_commit = False):
        self.db = mysql.connector.connect(
                        host = host,
                        port = port,
                        user = user,
                        password = password,
                        database = db_name)
        self.cursor = self.db.cursor()
        self.auto_commit = auto_commit
    
    
    def create_table(self, name:str = None, field:dict = None):
        lines = ''
        for n, (k, v) in enumerate(field.items()):
            if n != 0 :
                lines = lines + ',\n'
            line = f'{k} {v}'
            lines = lines + line
        sql = f"""CREATE TABLE {name} (\n{lines}\n);"""
        try :
            print(sql)
            self.cursor.execute(sql)
            print('done')
        except Exception as e:
            print(e)
    
    
    def drop_table(self, name:str = None):
        sql = f"""DROP TABLE {name};"""
        try :
            print(sql)
            self.cursor.execute(sql)
            print('done')
        except Exception as e:
            print(e)
    
    def insert_data(self, table:str, columns:str = None, values:list = None):
        value = ', '.join(f'"{w}"' if type(w) == str else f'{w}' for w in values)
        sql = ''
        if columns == None:
            sql = f"""INSERT INTO {table} VALUES({value})"""
        else :
            sql = f"""INSERT INTO {table}({columns}) VALUES({value})"""
        try :
            self.cursor.execute(sql)
            if self.auto_commit :
                self.db.commit()
        except Exception as e:
            print(e)
    
    
    def read_data(self, feature:str, table:str, added:str = None):
        if added == None:
            sql = f'SELECT {feature} FROM {table}'
            print(sql)
            self.cursor.execute(sql)
            res = self.cursor.fetchall()
            return res
        else :
            sql = f'SELECT {feature} FROM {table} {added}'
            print(sql)
            self.cursor.execute(sql)
            res = self.cursor.fetchall()
            return res
                        
    
    def custom(self, sql):
        self.cursor.execute(sql)
        res = self.cursor.fetchall()
        return res
    
    
    def commit(self):
        self.db.commit()

In [17]:
def api_to_db(df) :
    for line in df.iloc :
        date = line.name.date().strftime('%Y-%m-%d')
        itemname = line['item']
        kindname = line['kind']
        ranks = line['ranks']
        county = line['county']
        price = line['price']
        
        values = [date, itemname, kindname, ranks, county, price]
        db.insert_data('vegeprice', values = values)

In [18]:
HOST = '118.153.231.38'
PORT = 3306
USER = 'jcwee'
PASSWORD = 'password'
DATABASE_NAME = 'vegetable'

In [19]:
db = MySQLDBCrud(HOST, PORT, USER, PASSWORD, DATABASE_NAME)

In [6]:
# db.create_table('vegeprice',
#                 {
#                     'date' : 'DATE',
#                     'itemname' : 'VARCHAR(20)',
#                     'kindname' : 'VARCHAR(20)',
#                     'ranks' : 'VARCHAR(20)',
#                     'county' : 'VARCHAR(20)',
#                     'price' : 'INT'
#                 })

CREATE TABLE vegeprice (
date DATE,
itemname VARCHAR(20),
kindname VARCHAR(20),
ranks VARCHAR(20),
county VARCHAR(20),
price INT
);
done


In [6]:
res = db.read_data('*', 'vegeprice')

SELECT * FROM vegeprice


In [7]:
pd.DataFrame(res).to_csv('./data/vegetable_price.csv', encoding = 'cp949')

In [43]:
res = db.read_data('price', 'vegeprice')
pd.DataFrame(res)

SELECT price FROM vegeprice


,0
0,34000
1,34000
2,34000
3,34000
4,34000
...,...
49891,31000
49892,31000
49893,30000
49894,30000
